In [1]:
from datetime import datetime as dt
from random import random as rd
from random import randint as rdint
import csv

In [2]:
def random_id(from_id=1,to_id=100):
    return str(rdint(from_id,to_id))

In [3]:
def random_login_status(success=85, failed=None, unknown=0.001):
    if failed == None:
        failed = 100-unknown-success
    if success+failed+unknown != 100:
        ValueError('Login error rate not add up to 100%')
    login_dice = rd()*100
    if login_dice <= unknown:
        return 'UNKNOWN'
    elif login_dice <= failed:
        return 'FAILED'
    else:
        return 'SUCCESS'

In [4]:
def random_date():
    dt1 = dt(2010,1,1,0,0,0).timestamp()
    dt2 = dt(2024,1,31,23,59,59).timestamp()
    return dt.strftime(dt.fromtimestamp(dt1 + ((dt2-dt1)*rd())),'%Y-%m-%d %H:%M:%S')

In [5]:
def random_ipv4():
    return '.'.join(('%d' % rdint(1,255) for i in range(4)))
def random_ipv6():
    return ':'.join(('%x' % rdint(0, 16**4) for i in range(8)))


In [6]:
def random_boolean(true_rate=None):
    if true_rate == None:
        true_rate = 50
    return (True if rdint(1,100) <= true_rate else False)

In [33]:
import psycopg2
con = psycopg2.connect(
    database='postgres',
    user='postgres',
    password='postgrepassword',
    host='localhost'
)
cur = con.cursor()
table_name = 'login_data'
cur.execute(f'select * from information_schema.tables where table_name=\'{table_name}\';')

if bool(cur.rowcount):
    print(f'Table {table_name} exist, dropping...')
    cur.execute(f'DROP TABLE {table_name};')

print(f'Creating table {table_name}...')
cur.execute(f'CREATE TABLE {table_name} (id serial PRIMARY KEY,  user_id varchar(5), login_time timestamp, ipv4 varchar(15), ipv6 varchar(50), status varchar(7));')

row_total = 2000000

print(f'Inserting {row_total} records, please wait...')
for i in range(row_total):
    vals = f'INSERT INTO {table_name} (user_id, login_time, ipv4, ipv6, status) VALUES (\'{str(random_id(1,999))}\',\'{random_date()}\',\'{str(random_ipv4())}\',\'{str(random_ipv6())}\',\'{str(random_login_status())}\');'
    # try:
    cur.execute(vals)
    # except:
    #     print(f'Error at: {vals}')
    if(i%10000 == 0 and i != 0):
        print(f'Inserted {i} rows...')

cur.execute(f'SELECT COUNT(*) FROM {table_name};')
print(f'Created {cur.fetchone()[0]} rows!')

cur.execute(f'SELECT COUNT(*) FROM {table_name} WHERE status = \'SUCCESS\';')
print(f'Total \'Success\' login: {cur.fetchone()[0]} rows!')

print('Committing...')
con.commit()
print('Committed!')
cur.close()
con.close()


Table login_data exist, dropping...
Creating table login_data...
Inserting 2000000 records, please wait...
